In [0]:
import pandas as pd
import numpy as np

In [0]:
df = pd.read_csv('https://raw.githubusercontent.com/albanda/CE888/master/lab5-recommender/jester-data-1.csv',header=None)

In [6]:
df.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100
0,74,-7.82,8.79,-9.66,-8.16,-7.52,-8.50,-9.85,4.17,-8.98,-4.76,-8.50,-6.75,-7.18,8.45,-7.18,-7.52,-7.43,-9.81,-9.85,-9.85,-9.37,1.50,-4.37,-9.81,-8.50,1.12,7.82,2.86,9.13,-7.43,2.14,-4.08,-9.08,7.82,5.05,4.95,-9.17,-8.40,-8.40,...,8.59,3.59,-6.84,-9.03,2.82,-1.36,-9.08,8.30,5.68,-4.81,99.0,99.00,99.0,99.00,99.00,99.00,99.00,-9.42,99.00,99.00,99.00,-7.72,99.00,99.00,99.00,99.00,99.00,99.00,99.00,99.00,2.82,99.00,99.00,99.00,99.00,99.00,-5.63,99.00,99.00,99.00
1,100,4.08,-0.29,6.36,4.37,-2.38,-9.66,-0.73,-5.34,8.88,9.22,6.75,8.64,4.42,7.43,4.56,-0.97,4.66,-0.68,3.30,-1.21,0.87,8.64,8.35,9.17,0.05,7.57,4.71,0.87,-0.39,6.99,6.50,-0.92,7.14,9.03,-1.80,0.73,7.09,3.40,-0.87,...,-6.70,-3.35,-9.03,4.47,4.08,-3.83,8.74,1.12,0.78,7.52,-5.0,2.77,8.3,7.77,7.33,6.21,7.72,8.98,8.64,8.20,3.93,4.85,4.85,6.07,8.98,4.51,-0.05,3.69,4.56,0.58,2.82,-4.95,-0.29,7.86,-0.19,-2.14,3.06,0.34,-4.32,1.07
2,49,99.00,99.00,99.00,99.00,9.03,9.27,9.03,9.27,99.00,99.00,7.33,7.57,9.37,6.17,-6.36,-6.89,-7.86,9.03,9.03,9.03,7.28,99.00,8.25,99.00,99.00,7.48,7.28,7.28,8.93,99.00,6.17,7.28,99.00,99.00,8.98,7.33,99.00,6.17,9.08,...,6.46,7.28,99.00,99.00,7.04,7.28,99.00,7.28,8.25,99.00,99.0,99.00,99.0,99.00,99.00,8.93,99.00,99.00,99.00,9.08,99.00,99.00,99.00,99.00,99.00,99.00,99.00,99.00,99.00,9.03,99.00,99.00,99.00,9.08,99.00,99.00,99.00,99.00,99.00,99.00
3,48,99.00,8.35,99.00,99.00,1.80,8.16,-2.82,6.21,99.00,1.84,7.33,6.60,6.31,8.11,-7.23,-6.65,1.17,-6.60,-3.64,-2.09,5.34,99.00,99.00,99.00,99.00,2.91,3.93,6.75,6.60,99.00,6.65,-6.12,99.00,7.57,6.21,6.65,99.00,-8.30,7.18,...,0.00,-3.69,99.00,99.00,7.82,0.24,99.00,7.28,-2.33,99.00,99.0,99.00,99.0,99.00,99.00,99.00,99.00,99.00,99.00,99.00,99.00,99.00,99.00,99.00,0.63,99.00,99.00,-2.33,99.00,99.00,99.00,99.00,99.00,0.53,99.00,99.00,99.00,99.00,99.00,99.00
4,91,8.50,4.61,-4.17,-5.39,1.36,1.60,7.04,4.61,-0.44,5.73,8.25,6.84,-3.93,7.23,-2.33,-9.66,2.72,-1.36,2.57,4.51,8.20,6.12,8.30,-1.26,7.77,1.89,-1.17,5.68,8.45,4.61,8.06,-9.47,7.28,5.68,2.48,3.20,-1.26,6.80,4.51,...,7.38,6.17,4.71,-2.28,7.38,4.56,7.14,4.22,3.01,3.83,99.0,99.00,99.0,99.00,99.00,99.00,4.13,99.00,99.00,99.00,5.24,5.92,0.87,7.28,3.93,-0.63,6.31,4.71,2.82,2.96,5.19,5.58,4.27,5.19,5.73,1.55,3.11,6.55,1.80,1.60


In [7]:
df.shape

(24983, 101)

In [0]:
def replace(orig, percentage=0.1):
  """
  Replaces 'percentage'% of the original values in 'orig' with 99's
  :param orig: original data array
  :param percentage: percentage of values to replace (0<percentage<1)
  """
  new_data = orig.copy()
  rated = np.where(arr!=99)
  n_rated = len(rated[0])
  idx = np.random.choice(n_rated, size=int(percentage*n_rated), replace=False)
  new_data[rated[0][idx], rated[1][idx]] = 99
  return new_data, (rated[0][idx], rated[1][idx])

In [0]:
arr = df.values

In [0]:
rated = np.where(arr!=99)

In [0]:
new_arr, validation_indices = replace(arr, 0.1)

In [0]:
n_latent_factors = 2

user_ratings = new_arr
# Initialise as random values
latent_user_preferences = np.random.random((new_arr.shape[0], n_latent_factors))
latent_item_features = np.random.random((new_arr.shape[1], n_latent_factors))

In [0]:
def predict_rating(user_id, item_id):
    """ Predict a rating given a user_id and an item_id.
    """
    user_preference = latent_user_preferences[user_id]
    item_preference = latent_item_features[item_id]
    return user_preference.dot(item_preference)


def train(user_id, item_id, rating, alpha=0.0001):
    
    #print item_id
    prediction_rating = predict_rating(user_id, item_id)
    err =  prediction_rating - rating
    #print err
    user_pref_values = latent_user_preferences[user_id][:]
    latent_user_preferences[user_id] -= alpha * err * latent_item_features[item_id]
    latent_item_features[item_id] -= alpha * err * user_pref_values
    return err
    

def sgd(iterations=50):
    """ Iterate over all users and all items and train for 
        a certain number of iterations
    """
    for iteration in range(iterations):
        error = []
        for user_id in range(latent_user_preferences.shape[0]):
            for item_id in range(latent_item_features.shape[0]):
                rating = user_ratings[user_id][item_id]
                if rating != 99:
                    err = train(user_id, item_id, rating)
                    error.append(err)
        mse = (np.array(error) ** 2).mean()   
        if (iteration % 10) == 0:
            print(mse)

In [17]:
sgd()

59.98238346710939
24.544827896628465
24.05287349073756
18.952112743423008
17.975434466730256


In [0]:
predictions = latent_user_preferences.dot(latent_item_features.T)
values = [zip(user_ratings[i], predictions[i]) for i in range(predictions.shape[0])]
comparison_data = pd.DataFrame(values)

In [26]:
comparison_data

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100
0,"(74.0, 77.59729153680217)","(-7.82, -2.771498378270078)","(8.79, -3.836114398114984)","(-9.66, -3.9009377505329685)","(-8.16, -6.126507637058779)","(-7.52, -2.6978901709669105)","(-8.5, -3.1337161372620663)","(-9.85, -3.3912923465150175)","(4.17, -2.4090011731876886)","(-8.98, -5.260502677993157)","(-4.76, -2.544123275864231)","(-8.5, -2.5841456248692416)","(-6.75, -3.4368932141240176)","(-7.18, -4.100889281988832)","(8.45, -3.4906439405656116)","(-7.18, -3.4468858559364386)","(-7.52, -5.055498701059476)","(-7.43, -3.4228642439716817)","(-9.81, -3.2637117502296507)","(-9.85, -3.1665126659858664)","(-9.85, -4.572278445655984)","(-9.37, -2.2707481513497347)","(1.5, -3.9016404766638395)","(-4.37, -4.357497087249008)","(-9.81, -6.035361351615251)","(99.0, -4.340650406631227)","(1.12, -3.4961508690724163)","(7.82, -0.23128936994833832)","(2.86, -2.953191428025405)","(9.13, -0.2952637745155099)","(99.0, -5.625387398793486)","(2.14, -1.9118012446890444)","(-4.08, -0.2889962266969503)","(-9.08, -6.040058428684801)","(7.82, -3.679700311252911)","(99.0, -0.6793558761450781)","(4.95, 0.11563802727105628)","(-9.17, -5.312797499555095)","(-8.4, -3.0842413924389565)","(-8.4, -3.7485347849442494)",...,"(8.59, -1.8625258523851915)","(3.59, -0.9343430160345987)","(-6.84, -4.453071052221755)","(-9.03, -4.89894438965313)","(2.82, -1.8648220841478496)","(-1.36, -1.4695342143253063)","(-9.08, -5.339534956578855)","(8.3, -1.2194843659393833)","(5.68, -1.4625394483682257)","(-4.81, -4.142614480915092)","(99.0, -4.616478170512555)","(99.0, -1.5174241048078883)","(99.0, -3.781164078583774)","(99.0, -5.66834782011204)","(99.0, -4.279864732644812)","(99.0, -1.606237149629212)","(99.0, -3.7793874816223303)","(-9.42, -3.0128601129365284)","(99.0, -4.588365722533514)","(99.0, -3.7635129085060917)","(99.0, -2.542672824398949)","(99.0, -3.5268552162708064)","(99.0, -2.250980203612966)","(99.0, -3.8169434437425944)","(99.0, -3.9270502368693223)","(99.0, -3.433609796433449)","(99.0, -2.189698856516436)","(99.0, -1.9050643643933407)","(99.0, -0.08903708684414426)","(99.0, -3.8782006524151758)","(2.82, -1.901571548655252)","(99.0, -3.2535705086147844)","(99.0, -1.6596664616286125)","(99.0, -2.923793617551931)","(99.0, -2.867712265750488)","(99.0, -2.66876761271732)","(-5.63, -2.3110603124734554)","(99.0, -3.49645599849238)","(99.0, -4.385310319074118)","(99.0, -3.355749595741739)"
1,"(100.0, 95.38632227835771)","(4.08, 2.772847141501291)","(-0.29, 2.328414871617673)","(6.36, 2.343186090880143)","(4.37, 0.6948306920174504)","(-2.38, 1.912957574773634)","(-9.66, 3.881681554228929)","(-0.73, 0.8447471997798602)","(-5.34, 0.10533331547123287)","(8.88, 1.4554756767911436)","(9.22, 3.1452014469157668)","(6.75, 4.047608346792779)","(99.0, 3.9348400285724714)","(99.0, -1.1367511468331144)","(7.43, 3.888982870381488)","(4.56, -1.3241167336897397)","(-0.97, -2.8419736607617483)","(4.66, -0.24067625247466973)","(-0.68, 0.3246353179135492)","(99.0, 1.6983223115893304)","(99.0, 0.44462417511226093)","(0.87, 4.641261821644967)","(8.64, 3.1855964170079236)","(8.35, 2.234684315956194)","(9.17, 0.24758065894098463)","(0.05, 2.6597562173222533)","(7.57, 3.7083365537367037)","(99.0, 5.455263419128965)","(0.87, 3.769243742357062)","(99.0, 5.124307668213285)","(99.0, 1.7728662705330185)","(6.5, 4.581588472267929)","(-0.92, 5.413550800112787)","(7.14, 0.6661186015682297)","(9.03, 3.21110952568105)","(-1.8, 5.347196125046178)","(0.73, 5.411606441928498)","(7.09, 0.3485619576274355)","(3.4, 3.4053005208527254)","(-0.87, 3.5013278065738027)",...,"(-6.7, 4.896382175166535)","(-3.35, 5.309693861003982)","(-9.03, 2.2741402179545314)","(4.47, 1.1810921129847662)","(4.08, 4.684771634072931)","(-3.83, 4.828186631079244)","(8.74, 1.0984956167738835)","(1.12, 

In [0]:
original_data = arr
assert len(validation_indices[0]) == len(validation_indices[1])
validation_size = len(validation_indices[0])
errors = []
for i in range(validation_size):
  user_id = validation_indices[0][i]
  joke_id = validation_indices[1][i]
  prediction_rating = predict_rating(user_id, joke_id)
  real_rating = original_data[user_id][joke_id]
  error = prediction_rating - real_rating
  errors.append(error)
mse = (np.array(errors) ** 2).mean()

In [0]:
filled_data = np.zeros(original_data.shape)
rows, columns = original_data.shape
for user_id in range(rows):
  for joke_id in range(columns):
    if original_data[user_id][joke_id] == 99:
      filled_data[user_id][joke_id] = predict_rating(user_id, joke_id)
    else:
      filled_data[user_id][joke_id] = original_data[user_id][joke_id]

In [34]:
print(filled_data)

[[ 74.          -7.82         8.79       ...  -3.496456    -4.38531032
   -3.3557496 ]
 [100.           4.08        -0.29       ...   0.34        -4.32
    1.07      ]
 [ 49.           6.59508578   7.09065243 ...   7.27912793   6.94563791
    7.9514426 ]
 ...
 [ 39.          -0.11124134  -0.28857935 ...  -0.20910742  -0.40652692
   -0.13693325]
 [ 37.          -0.61795499  -1.03851845 ...  -0.87318455  -1.29148875
   -0.75127466]
 [ 72.           2.43         2.67       ...   3.32548418   2.37349493
    3.98604696]]


In [0]:
means = np.mean(filled_data, axis=0)

In [0]:
max_joke = -9999999
max_joke_index = -1
min_joke = 99999999
min_joke_index = -1


In [0]:

for i in range(len(means)):
  if means[i] > max_joke:
    max_joke = means[i]
    max_joke_i = i
  elif means[i] < min_joke:
    min_joke = means[i]
    min_joke_i = i


In [37]:
print(f'best joke {max_joke_index + 1}, rating : {max_joke}')
print(f'worst joke {min_joke_index + 1},rating: {min_joke}')

best joke 1, rating : 72.36418906437233
worst joke 59,rating: -3.164247013214021
